In [8]:
import numpy as np 
import pandas as pd
import os 
import sys
from distutils.dir_util import copy_tree

# go to the directory where utility functions are stored
os.chdir(r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\ModelAssembling')
from BuildingModelClass import BuildingModel
from utils import *
import subprocess
import json
from json import JSONEncoder
import subprocess

import time

In [2]:
# Utility function directory 
UtilDirectory = r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\ModelAssembling'
# Base directory is the main directory that models, model inputs and utility directory stores
BaseDirectory = UtilDirectory.replace("ModelAssembling", "")
# Model directory is where you want to store your model
ModelDirectory = BaseDirectory + '\BuildingModels'
# DB directory is where you store Database.csv (for steel section)
DBDirecctory = UtilDirectory

# If there is no model directory, create one
if os.path.isdir(ModelDirectory) != True:
    os.chdir(BaseDirectory)
    os.mkdir('BuildingModels')
    
# Read in building name 
# Make sure the building name is consistent with the input folders
os.chdir(UtilDirectory)
with open('BuildingName.txt', 'r') as f:
    BuildingList = f.read() 
BuildingList = BuildingList.split('\n')

# Generate eigen analysis, pushover analysis and dynamic analysis models for each one in the building list
for i in range(len(BuildingList)):
    InfoDirectory = BaseDirectory + '\BuildingInfo\%s'%BuildingList[i]
    ModelClass = BuildingModel(BuildingList[i], InfoDirectory)

    # alternatively, one can choose to read inputs from json 
    ModelClass.read_in_txt_inputs(BuildingList[i], InfoDirectory)

    if os.path.isdir(ModelDirectory+'\%s'%BuildingList[i]) != True:
        os.chdir(ModelDirectory)
        os.mkdir('%s'%BuildingList[i])
        
    os.chdir(ModelDirectory+'/%s'%BuildingList[i])
    period = generateModalAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory)
    # Turn off RunPushoverSwitch to speed up the model creation 
    generatePushoverAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory,
                               GenerateModelSwitch = True, RunPushoverSwitch = False)
    generateDynamicAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory, period,
                               GenerateModelSwitch = True)
    print(period)


[0.3522614619208698, 0.25412715353010734, 0.229149425056406, 0.10394190316450584]
[0.3566760968107755, 0.26246152581949644, 0.229493709707616, 0.10406880260602897]


## Single Scale Single GM Analysis

In [10]:
Scale_Sa_GM = '0.162958971	0.325917943	0.488876914	0.651835885	0.814794857	0.977753828	1.140712799	1.303671771	1.466630742	1.629589713	1.792548685	1.955507656	2.118466627	2.281425599	2.44438457'
GM_Num = '22 22 22 22 22 22 22 22 22 22 22 22 22 22 22'
GM_ID = 1 # GM pair
GM_folder = r'GM_sets/FEMA44_FarFault'
Model_Name = BuildingList[0]


s = time.time()
SetupDyamaicAnalysis(ModelDirectory, Scale_Sa_GM, GM_Num, GM_ID, GM_folder, Model_Name, 1)
os.chdir(r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\BuildingModels')
r = os.system('OpenSees RunDynamic_Single.tcl')
f = time.time()
if not r: 
    print('Hazard Level %i GM Pair %s has finished successfully in %.3fs!'%(int(GM_ID/22)+1, str(GM_ID-int(GM_ID/22)), f-s))
else: print('GM Pair %s has failed'%str(GM_ID))

Hazard Level 0 GM Pair 1 has finished successfully in 13.043s!


## Incremental Dynamic Analysis 

In [15]:
Scale_Sa_GM = '0.162958971	0.325917943	0.488876914	0.651835885	0.814794857	0.977753828	1.140712799	1.303671771	1.466630742	1.629589713	1.792548685	1.955507656	2.118466627	2.281425599	2.44438457'
GM_Num = '22 22 22 22 22 22 22 22 22 22 22 22 22 22 22'
GM_folder = r'GM_sets/FEMA44_FarFault'
Model_Name = BuildingList[0]

# for FEMA Far Field GM set, it contains 22 pairs, we use 15 hazard levels for our IDA
start_ID, finish_ID = 1, 23
acc_time = 0

# Pairing ID == 1
for GM_ID in range(start_ID, finish_ID):
    s = time.time()
    SetupDyamaicAnalysis(ModelDirectory, Scale_Sa_GM, GM_Num, GM_ID, GM_folder, Model_Name, 1)
    os.chdir(r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\BuildingModels')
    r = os.system('OpenSees RunDynamic_Single.tcl')
    f = time.time()
    if not r: 
        print('Hazard Level %i GM Pair %s with Pairing ID %i has finished successfully in %.3fs!'%(int(GM_ID/22)+1, str(GM_ID-int(GM_ID/22)), 1, f-s))
        os.remove('RunDynamic_Single.tcl')
        acc_time += (f-s)
        print('Estimate remaining time %.3fs!'%(acc_time/(GM_ID - start_ID)*(finish_ID - GM_ID))
    else: 
        print('GM Pair %s has failed'%str(GM_ID))
        break

# Pairing ID == 2
for GM_ID in range(start_ID, finish_ID):
    s = time.time()
    SetupDyamaicAnalysis(ModelDirectory, Scale_Sa_GM, GM_Num, GM_ID, GM_folder, Model_Name, 1)
    os.chdir(r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\BuildingModels')
    r = os.system('OpenSees RunDynamic_Single.tcl')
    f = time.time()
    if not r: 
        print('Hazard Level %i GM Pair %s with Pairing ID %i has finished successfully in %.3fs!'%(int(GM_ID/22)+1, str(GM_ID-int(GM_ID/22)), 1, f-s))
        os.remove('RunDynamic_Single.tcl')
        acc_time += (f-s)
        print('Estimate remaining time %.3fs!'%(acc_time/(GM_ID - start_ID)*(finish_ID - GM_ID))
    else: 
        print('GM Pair %s has failed'%str(GM_ID))
        break

Hazard Level 1 GM Pair 1 with Pairing ID 1 has finished successfully in 12.703s!
Estimate remaining time 266.763
Hazard Level 1 GM Pair 2 with Pairing ID 1 has finished successfully in 9.156s!
Estimate remaining time 183.127
Hazard Level 1 GM Pair 3 with Pairing ID 1 has finished successfully in 23.016s!
Estimate remaining time 437.297
Hazard Level 1 GM Pair 4 with Pairing ID 1 has finished successfully in 18.352s!
Estimate remaining time 330.339
Hazard Level 1 GM Pair 5 with Pairing ID 1 has finished successfully in 40.631s!
Estimate remaining time 690.719
Hazard Level 1 GM Pair 6 with Pairing ID 1 has finished successfully in 30.533s!
Estimate remaining time 488.521
Hazard Level 1 GM Pair 7 with Pairing ID 1 has finished successfully in 24.115s!
Estimate remaining time 361.724
Hazard Level 1 GM Pair 8 with Pairing ID 1 has finished successfully in 15.419s!
Estimate remaining time 215.864
Hazard Level 1 GM Pair 9 with Pairing ID 1 has finished successfully in 19.765s!
Estimate remaini